In [1]:
import sys
print('python', sys.version)

import numpy as np
print('numpy', np.__version__)

import pandas as pd
print('pandas', pd.__version__)

import matplotlib as mpl
print('matplotlib', mpl.__version__)

import matplotlib.pyplot as plt
plt.rc("font", family="Malgun Gothic") # 한글표시 (window)
plt.rc("axes", unicode_minus=False) # x,y축 (-)부호 표시

# 레티나 디스플레이로 폰트가 선명하게 표시되도록 합니다.
from IPython.display import set_matplotlib_formats
set_matplotlib_formats("retina")

import seaborn as sns
print('seaborn', sns.__version__)


# 결과 확인을 용이하게 하기 위한 코드
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

# 모든 컬럼이 표시되도록 max_columns 의 수를 지정합니다.
pd.options.display.max_columns = 39

python 3.8.5 (default, May 27 2021, 13:30:53) 
[GCC 9.3.0]
numpy 1.19.5
pandas 1.2.5
matplotlib 3.4.2


<ipython-input-1-b0b59554c2c7>:19: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  set_matplotlib_formats("retina")


seaborn 0.11.1


# data read

In [2]:
lab = pd.read_csv('/project/data/폐렴환자lab.csv')
patient = pd.read_csv('/project/data/폐렴환자.csv')

# 전처리

## patient 전처리

### 환자 7799 맞춰주기

In [3]:
all = set(patient['SUBJECT_ID']).intersection(set(lab['SUBJECT_ID']))
set(patient['SUBJECT_ID'].unique())-all

{9388, 17674, 19097, 48968, 58012, 60961, 93114, 95230}

In [4]:
patient = patient[patient['SUBJECT_ID'].isin(all)]
patient

,ROW_ID,SUBJECT_ID,GENDER,DOB,DOD,DOD_HOSP,DOD_SSN,EXPIRE_FLAG
0,237,252,M,2078-03-06 00:00:00,NaN,NaN,NaN,0
1,238,253,F,2089-11-26 00:00:00,NaN,NaN,NaN,0
2,251,268,F,2132-02-21 00:00:00,2198-02-18 00:00:00,2198-02-18 00:00:00,NaN,1
3,252,269,M,2130-09-30 00:00:00,NaN,NaN,NaN,0
4,628,665,M,2052-05-20 00:00:00,2120-02-04 00:00:00,2120-02-04 00:00:00,NaN,1
...,...,...,...,...,...,...,...,...
7802,31801,43932,M,2080-06-26 00:00:00,2153-03-01 00:00:00,NaN,2153-03-01 00:00:00,1
7803,31802,43937,M,2113-03-30 00:00:00,2200-08-05 00:00:00,2200-08-05 00:00:00,2200-08-05 00:00:00,1
7804,31807,43959,F,2028-05-13 00:00:00,NaN,NaN,NaN,0
7805,31813,43982,F,2065-04-17 00:00:00,NaN,NaN,NaN,0


### 시간 부분 제거, 날짜만 남기기

In [5]:
import datetime as dt

patient['DOD'] = pd.to_datetime(patient['DOD'])
patient['DOD'] = patient['DOD'].dt.date
patient

,ROW_ID,SUBJECT_ID,GENDER,DOB,DOD,DOD_HOSP,DOD_SSN,EXPIRE_FLAG
0,237,252,M,2078-03-06 00:00:00,NaT,NaN,NaN,0
1,238,253,F,2089-11-26 00:00:00,NaT,NaN,NaN,0
2,251,268,F,2132-02-21 00:00:00,2198-02-18,2198-02-18 00:00:00,NaN,1
3,252,269,M,2130-09-30 00:00:00,NaT,NaN,NaN,0
4,628,665,M,2052-05-20 00:00:00,2120-02-04,2120-02-04 00:00:00,NaN,1
...,...,...,...,...,...,...,...,...
7802,31801,43932,M,2080-06-26 00:00:00,2153-03-01,NaN,2153-03-01 00:00:00,1
7803,31802,43937,M,2113-03-30 00:00:00,2200-08-05,2200-08-05 00:00:00,2200-08-05 00:00:00,1
7804,31807,43959,F,2028-05-13 00:00:00,NaT,NaN,NaN,0
7805,31813,43982,F,2065-04-17 00:00:00,NaT,NaN,NaN,0


### columns 선택 , 인덱스 정리 , 정렬

In [6]:
patient_sorted = patient[['SUBJECT_ID', 'EXPIRE_FLAG']].set_index('SUBJECT_ID').sort_values(['SUBJECT_ID'])#, inplace = True)
patient_sorted

,EXPIRE_FLAG
SUBJECT_ID,
9,1
21,1
31,1
36,0
37,1
...,...
99863,0
99934,0
99937,1


## lab 전처리

### lab columns 선택, 행 중복제거, 시간 부분 제거, 날짜만 남기기, 열 정렬

In [7]:
lab

lab_sorted_ = lab[['SUBJECT_ID', 'CHARTTIME', 'ITEMID', 'FLAG']].drop_duplicates()
lab_sorted_

import datetime as dt
lab_sorted_['CHARTTIME'] = pd.to_datetime(lab_sorted_['CHARTTIME'])
lab_sorted_['CHARTTIME'] = lab_sorted_['CHARTTIME'].dt.date
lab_sorted_

lab_sorted = lab_sorted_.set_index('SUBJECT_ID').sort_values(['SUBJECT_ID', 'CHARTTIME'])

,ROW_ID,SUBJECT_ID,HADM_ID,ITEMID,CHARTTIME,FLAG
0,5144,9,150750.0,50819,2149-11-14 06:07:00,NaN
1,5145,9,150750.0,50820,2149-11-14 06:07:00,NaN
2,5146,9,150750.0,50821,2149-11-14 06:07:00,abnormal
3,5147,9,150750.0,50826,2149-11-14 06:07:00,NaN
4,5148,9,150750.0,50827,2149-11-14 06:07:00,NaN
...,...,...,...,...,...,...
9356344,27427231,96442,120151.0,51248,2115-06-29 04:45:00,NaN
9356345,27427232,96442,120151.0,51249,2115-06-29 04:45:00,NaN
9356346,27427233,96442,120151.0,51250,2115-06-29 04:45:00,NaN
9356347,27427234,96442,120151.0,51254,2115-06-29 04:45:00,abnormal


,SUBJECT_ID,CHARTTIME,ITEMID,FLAG
0,9,2149-11-14 06:07:00,50819,NaN
1,9,2149-11-14 06:07:00,50820,NaN
2,9,2149-11-14 06:07:00,50821,abnormal
3,9,2149-11-14 06:07:00,50826,NaN
4,9,2149-11-14 06:07:00,50827,NaN
...,...,...,...,...
9356344,96442,2115-06-29 04:45:00,51248,NaN
9356345,96442,2115-06-29 04:45:00,51249,NaN
9356346,96442,2115-06-29 04:45:00,51250,NaN
9356347,96442,2115-06-29 04:45:00,51254,abnormal


,SUBJECT_ID,CHARTTIME,ITEMID,FLAG
0,9,2149-11-14,50819,NaN
1,9,2149-11-14,50820,NaN
2,9,2149-11-14,50821,abnormal
3,9,2149-11-14,50826,NaN
4,9,2149-11-14,50827,NaN
...,...,...,...,...
9356344,96442,2115-06-29,51248,NaN
9356345,96442,2115-06-29,51249,NaN
9356346,96442,2115-06-29,51250,NaN
9356347,96442,2115-06-29,51254,abnormal


### abnormal -> 1
### NaN -> 0

In [8]:
lab_sorted['FLAG'].replace('abnormal', 1, inplace = True) #abnormal을 1로 바꿔주는 코드

lab_sorted[lab_sorted.index==9]['CHARTTIME'].max() #lab기록의 마지막 날짜 DOD랑 같은지 확인하려고 친 코드

datetime.date(2149, 11, 14)

In [9]:
lab_sorted.fillna(0, inplace = True)

In [10]:
lab_sorted['CHARTTIME'].unique()[-10:]

array([datetime.date(2155, 12, 12), datetime.date(2194, 7, 24),
       datetime.date(2194, 7, 26), datetime.date(2194, 7, 27),
       datetime.date(2126, 5, 3), datetime.date(2195, 4, 23),
       datetime.date(2181, 4, 7), datetime.date(2171, 7, 30),
       datetime.date(2171, 7, 31), datetime.date(2186, 4, 17)],
      dtype=object)

In [11]:
lab_sorted.index.unique()

Int64Index([    9,    21,    31,    36,    37,    41,    68,    85,    94,
              101,
            ...
            99666, 99781, 99817, 99822, 99836, 99863, 99934, 99937, 99944,
            99985],
           dtype='int64', name='SUBJECT_ID', length=7799)

### lab과 patient merge

In [12]:
#expire_flag는 y(label)값. 함께 보려고 lab과 patient 쓰는 것만 붙여줌.
merge_lab_pat = pd.merge(left=lab_sorted, right= patient_sorted, how='inner', left_index=True,right_index=True)

In [13]:
merge_lab_pat

,CHARTTIME,ITEMID,FLAG,EXPIRE_FLAG
SUBJECT_ID,,,,
9,2149-11-09,50856,0,1
9,2149-11-09,50868,0,1
9,2149-11-09,50879,0,1
9,2149-11-09,50880,0,1
9,2149-11-09,50882,1,1
...,...,...,...,...
99985,2182-03-14,51006,1,0
99985,2182-03-14,51146,0,0
99985,2182-03-14,51200,0,0


In [14]:
# value가 그냥 들어있음.
dic = {}
for i in lab_sorted.index.unique():
    dic[i] = lab_sorted[lab_sorted.index==i]['CHARTTIME'].unique()[-10:]
#     lab_sorted[lab_sorted.index==i]['CHARTTIME'].unique()

In [15]:
dic

{9: array([datetime.date(2149, 11, 9), datetime.date(2149, 11, 10),
        datetime.date(2149, 11, 11), datetime.date(2149, 11, 12),
        datetime.date(2149, 11, 13), datetime.date(2149, 11, 14)],
       dtype=object),
 21: array([datetime.date(2135, 1, 10), datetime.date(2135, 1, 11),
        datetime.date(2135, 1, 30), datetime.date(2135, 1, 31),
        datetime.date(2135, 2, 1), datetime.date(2135, 2, 2),
        datetime.date(2135, 2, 3), datetime.date(2135, 2, 4),
        datetime.date(2135, 2, 5), datetime.date(2135, 2, 6)], dtype=object),
 31: array([datetime.date(2108, 8, 22), datetime.date(2108, 8, 23),
        datetime.date(2108, 8, 24), datetime.date(2108, 8, 25),
        datetime.date(2108, 8, 26), datetime.date(2108, 8, 27),
        datetime.date(2108, 8, 28), datetime.date(2108, 8, 29),
        datetime.date(2108, 8, 30)], dtype=object),
 36: array([datetime.date(2134, 5, 11), datetime.date(2134, 5, 12),
        datetime.date(2134, 5, 13), datetime.date(2134, 5, 14),

# 교수님 피드백

In [16]:
df = merge_lab_pat.copy()
df

,CHARTTIME,ITEMID,FLAG,EXPIRE_FLAG
SUBJECT_ID,,,,
9,2149-11-09,50856,0,1
9,2149-11-09,50868,0,1
9,2149-11-09,50879,0,1
9,2149-11-09,50880,0,1
9,2149-11-09,50882,1,1
...,...,...,...,...
99985,2182-03-14,51006,1,0
99985,2182-03-14,51146,0,0
99985,2182-03-14,51200,0,0


In [17]:
dic_sub2idx = {}
for i,j in enumerate(df.index.unique()):
    dic_sub2idx[j] = i
dic_sub2idx

{9: 0,
 21: 1,
 31: 2,
 36: 3,
 37: 4,
 41: 5,
 68: 6,
 85: 7,
 94: 8,
 101: 9,
 103: 10,
 105: 11,
 109: 12,
 111: 13,
 112: 14,
 117: 15,
 124: 16,
 136: 17,
 148: 18,
 156: 19,
 157: 20,
 164: 21,
 166: 22,
 175: 23,
 177: 24,
 184: 25,
 186: 26,
 188: 27,
 191: 28,
 203: 29,
 208: 30,
 211: 31,
 214: 32,
 221: 33,
 222: 34,
 223: 35,
 224: 36,
 225: 37,
 231: 38,
 242: 39,
 248: 40,
 252: 41,
 253: 42,
 268: 43,
 269: 44,
 274: 45,
 283: 46,
 286: 47,
 290: 48,
 303: 49,
 305: 50,
 308: 51,
 323: 52,
 328: 53,
 330: 54,
 346: 55,
 356: 56,
 357: 57,
 360: 58,
 364: 59,
 366: 60,
 368: 61,
 370: 62,
 391: 63,
 404: 64,
 412: 65,
 423: 66,
 425: 67,
 426: 68,
 433: 69,
 434: 70,
 450: 71,
 452: 72,
 453: 73,
 457: 74,
 458: 75,
 481: 76,
 482: 77,
 492: 78,
 497: 79,
 499: 80,
 505: 81,
 507: 82,
 508: 83,
 509: 84,
 510: 85,
 514: 86,
 518: 87,
 530: 88,
 536: 89,
 538: 90,
 544: 91,
 546: 92,
 550: 93,
 565: 94,
 571: 95,
 576: 96,
 580: 97,
 586: 98,
 592: 99,
 596: 100,
 605: 101

In [18]:
len(df['ITEMID'].sort_values().unique())

690

In [19]:
dic_item2idx = {}
for i,j in enumerate(df['ITEMID'].sort_values().unique()):
    dic_item2idx[j] = i
dic_item2idx

{50800: 0,
 50801: 1,
 50802: 2,
 50803: 3,
 50804: 4,
 50805: 5,
 50806: 6,
 50808: 7,
 50809: 8,
 50810: 9,
 50811: 10,
 50812: 11,
 50813: 12,
 50814: 13,
 50815: 14,
 50816: 15,
 50817: 16,
 50818: 17,
 50819: 18,
 50820: 19,
 50821: 20,
 50822: 21,
 50823: 22,
 50824: 23,
 50825: 24,
 50826: 25,
 50827: 26,
 50828: 27,
 50829: 28,
 50830: 29,
 50831: 30,
 50832: 31,
 50835: 32,
 50836: 33,
 50837: 34,
 50838: 35,
 50839: 36,
 50840: 37,
 50841: 38,
 50842: 39,
 50843: 40,
 50844: 41,
 50845: 42,
 50847: 43,
 50848: 44,
 50849: 45,
 50850: 46,
 50851: 47,
 50852: 48,
 50853: 49,
 50854: 50,
 50855: 51,
 50856: 52,
 50857: 53,
 50858: 54,
 50859: 55,
 50860: 56,
 50861: 57,
 50862: 58,
 50863: 59,
 50864: 60,
 50865: 61,
 50866: 62,
 50867: 63,
 50868: 64,
 50869: 65,
 50870: 66,
 50871: 67,
 50872: 68,
 50873: 69,
 50874: 70,
 50875: 71,
 50876: 72,
 50877: 73,
 50878: 74,
 50879: 75,
 50880: 76,
 50881: 77,
 50882: 78,
 50883: 79,
 50884: 80,
 50885: 81,
 50886: 82,
 50887: 83,
 5

In [20]:
# 실행 금지
# for i in df.reset_index()["CHARTTIME"]:
#     df.reset_index()[["SUBJECT_ID", "CHARTTIME"]][df.reset_index()["CHARTTIME"]==i]

,SUBJECT_ID,CHARTTIME
0,9,2149-11-09
1,9,2149-11-09
2,9,2149-11-09
3,9,2149-11-09
4,9,2149-11-09
...,...,...
7302230,47816,2149-11-09
7302231,47816,2149-11-09
7302232,47816,2149-11-09
7302233,47816,2149-11-09


,SUBJECT_ID,CHARTTIME
0,9,2149-11-09
1,9,2149-11-09
2,9,2149-11-09
3,9,2149-11-09
4,9,2149-11-09
...,...,...
7302230,47816,2149-11-09
7302231,47816,2149-11-09
7302232,47816,2149-11-09
7302233,47816,2149-11-09


,SUBJECT_ID,CHARTTIME
0,9,2149-11-09
1,9,2149-11-09
2,9,2149-11-09
3,9,2149-11-09
4,9,2149-11-09
...,...,...
7302230,47816,2149-11-09
7302231,47816,2149-11-09
7302232,47816,2149-11-09
7302233,47816,2149-11-09


,SUBJECT_ID,CHARTTIME
0,9,2149-11-09
1,9,2149-11-09
2,9,2149-11-09
3,9,2149-11-09
4,9,2149-11-09
...,...,...
7302230,47816,2149-11-09
7302231,47816,2149-11-09
7302232,47816,2149-11-09
7302233,47816,2149-11-09


,SUBJECT_ID,CHARTTIME
0,9,2149-11-09
1,9,2149-11-09
2,9,2149-11-09
3,9,2149-11-09
4,9,2149-11-09
...,...,...
7302230,47816,2149-11-09
7302231,47816,2149-11-09
7302232,47816,2149-11-09
7302233,47816,2149-11-09


...

,SUBJECT_ID,CHARTTIME
0,9,2149-11-09
1,9,2149-11-09
2,9,2149-11-09
3,9,2149-11-09
4,9,2149-11-09
...,...,...
7302230,47816,2149-11-09
7302231,47816,2149-11-09
7302232,47816,2149-11-09
7302233,47816,2149-11-09


,SUBJECT_ID,CHARTTIME
0,9,2149-11-09
1,9,2149-11-09
2,9,2149-11-09
3,9,2149-11-09
4,9,2149-11-09
...,...,...
7302230,47816,2149-11-09
7302231,47816,2149-11-09
7302232,47816,2149-11-09
7302233,47816,2149-11-09


,SUBJECT_ID,CHARTTIME
0,9,2149-11-09
1,9,2149-11-09
2,9,2149-11-09
3,9,2149-11-09
4,9,2149-11-09
...,...,...
7302230,47816,2149-11-09
7302231,47816,2149-11-09
7302232,47816,2149-11-09
7302233,47816,2149-11-09


,SUBJECT_ID,CHARTTIME
0,9,2149-11-09
1,9,2149-11-09
2,9,2149-11-09
3,9,2149-11-09
4,9,2149-11-09
...,...,...
7302230,47816,2149-11-09
7302231,47816,2149-11-09
7302232,47816,2149-11-09
7302233,47816,2149-11-09


KeyboardInterrupt: 

In [21]:
len(df.reset_index()[["SUBJECT_ID","CHARTTIME"]])

9350027

In [43]:
df_sub_cht = df.reset_index()[["SUBJECT_ID","CHARTTIME"]].drop_duplicates()
df_sub_cht
df_sub_cht = df_sub_cht.reset_index()
df_sub_cht = df_sub_cht.drop('index', axis = 1)
df_sub_cht

,SUBJECT_ID,CHARTTIME
0,9,2149-11-09
94,9,2149-11-10
186,9,2149-11-11
234,9,2149-11-12
274,9,2149-11-13
...,...,...
9349903,99985,2181-02-12
9349923,99985,2181-02-16
9349950,99985,2181-02-22
9349978,99985,2181-03-06


,SUBJECT_ID,CHARTTIME
0,9,2149-11-09
1,9,2149-11-10
2,9,2149-11-11
3,9,2149-11-12
4,9,2149-11-13
...,...,...
235950,99985,2181-02-12
235951,99985,2181-02-16
235952,99985,2181-02-22
235953,99985,2181-03-06


In [69]:
# 이 경우 튜플리스트 부터 만들고 idx는 나중에 붙여.
# df_sub_cht = df.reset_index()[["SUBJECT_ID","CHARTTIME"]].drop_duplicates()
df_sub_cht = df.reset_index()[["SUBJECT_ID","CHARTTIME"]].drop_duplicates()
df_sub_cht = df_sub_cht.reset_index()
df_sub_cht = df_sub_cht.drop('index', axis = 1)

t = []
dic_sub_charttime2idx = {}

# 튜플리스트 만들기
for i in range(len(df_sub_cht)):
    t.append((df_sub_cht.iloc[i][0], df_sub_cht.iloc[i][1]))
t

#     break
#     dic_sub_charttime2idx[(k,j)] = i
# dic_sub_charttime2idx

[(9, datetime.date(2149, 11, 9)),
 (9, datetime.date(2149, 11, 10)),
 (9, datetime.date(2149, 11, 11)),
 (9, datetime.date(2149, 11, 12)),
 (9, datetime.date(2149, 11, 13)),
 (9, datetime.date(2149, 11, 14)),
 (21, datetime.date(2134, 8, 20)),
 (21, datetime.date(2134, 8, 25)),
 (21, datetime.date(2134, 9, 11)),
 (21, datetime.date(2134, 9, 12)),
 (21, datetime.date(2134, 9, 13)),
 (21, datetime.date(2134, 9, 14)),
 (21, datetime.date(2134, 9, 15)),
 (21, datetime.date(2134, 9, 16)),
 (21, datetime.date(2134, 9, 17)),
 (21, datetime.date(2134, 9, 18)),
 (21, datetime.date(2134, 9, 19)),
 (21, datetime.date(2134, 9, 20)),
 (21, datetime.date(2134, 9, 21)),
 (21, datetime.date(2134, 9, 22)),
 (21, datetime.date(2134, 9, 23)),
 (21, datetime.date(2134, 9, 24)),
 (21, datetime.date(2134, 9, 28)),
 (21, datetime.date(2134, 9, 29)),
 (21, datetime.date(2134, 9, 30)),
 (21, datetime.date(2134, 10, 1)),
 (21, datetime.date(2134, 12, 14)),
 (21, datetime.date(2134, 12, 22)),
 (21, datetime.date

In [77]:
lst = [1,2,3,3,4,5,6,7]
lst.count(3)

2

In [84]:
lst = []
tp = []
for i,j in enumerate(t):
#     i,j
    if len(lst) < 10 or lst.count(j[0])<=10:
        lst.append(t[i][0])
        t[i]
#         tp.append(t[i])
    elif len(lst) > 10 or lst.count(j[0]) > 10:
        lst = []

(9, datetime.date(2149, 11, 9))

(9, datetime.date(2149, 11, 10))

(9, datetime.date(2149, 11, 11))

(9, datetime.date(2149, 11, 12))

(9, datetime.date(2149, 11, 13))

...

(3092, datetime.date(2201, 5, 18))

(3092, datetime.date(2201, 5, 19))

(3092, datetime.date(2201, 5, 20))

(3092, datetime.date(2201, 5, 21))

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[99985, 99985, 99985, 99985, 99985]

In [65]:
df_sub_cht[df_sub_cht["SUBJECT_ID"] == 9]#["SUBJECT_ID"] == 9

,SUBJECT_ID,CHARTTIME
0,9,2149-11-09
1,9,2149-11-10
2,9,2149-11-11
3,9,2149-11-12
4,9,2149-11-13
5,9,2149-11-14


In [68]:
cnt = 0
for j in df.index.unique():
    j
#     for k in range(len(df_sub_cht)):
    if cnt == 0:
        if df_sub_cht[df_sub_cht["SUBJECT_ID"] == j]["SUBJECT_ID"] == j:
            df_sub_cht[df_sub_cht["SUBJECT_ID"] == j]
    #         cnt += 1
    #     else:
    #         cnt = 0

#             (df_sub_cht.iloc[i][1])

9

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [82]:
len(t)
for k in range(len(t)):
    for i,j in enumerate(df.index.unique()):
        t[i][0]==

235955

In [39]:
for i,j in df.shape:
    df.iloc[[i,j]]

TypeError: cannot unpack non-iterable int object